## Section 02: Monte Carlo Probabilities
**Abstract:** This notebook uses the data for several thousand poker hands in order to estimate certain probabilities, including winning probabilities for certain hole cards, draw "hits", and draw completion. The data (if not already available) can be sourced by fully executing [Section 01](./01_data_generation.ipynb). The results of each experiment can finally be stored as separate tables for use in data exploration or strategy development. 

In [1]:
# Imports
import pandas as pd
import numpy as np
from deuces import Card, Deck, Evaluator
import random

**Hole Cards Winning Probabilities**

The following chart describes the many probabilities of winning with certain hole cards,assuming a heads-up (1-opponent) situation. The upper triangle shows suited cards while the lower shows unsuited.

![Pre-Flop Probabilities Chart](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fimages.actionnetwork.com%2Fblog%2F2020%2F05%2FScreen-Shot-2020-05-25-at-12.00.45-PM.png&f=1&nofb=1&ipt=2cdde30e9afa0f00f8cb42dd3219c006c3b8f2bfce75d6312cf993a3273caa1c)

While this information is useful for early ideation, it does not represent probabilities that consider whether the number of opponents. More specifically, the desired probability can be formulated as such:
$$
P(Win | Rank1,Rank2,Suited,Players)
$$
By considering each possible combination of conditions and using the simulated data, these probabilities can be accurately estimated.

In [2]:
rows = []
for r1 in range(13): # Ranks from 0 (2) to 12 (A)
    for r2 in range(r1+1): # ranks from 0 to current rank
        for opponents in range(1,9):  # Number of opponents range from 1-8
            rows.append({
                'rank_set': (r2, r1),
                'suited': 0,
                'opponents': opponents
            })
            if r1 != r2: # possibility of suited exists
                rows.append({
                'rank_set': (r2, r1),
                'suited': 1,
                'opponents': opponents
            })
            
# create a dataframe using all of these rows
preflop_df = pd.DataFrame(rows)

# display heads_up data by filtering for opponents == 1
preflop_df[preflop_df['opponents'] == 1]

,rank_set,suited,opponents
0,"(0, 0)",0,1
8,"(0, 1)",0,1
9,"(0, 1)",1,1
24,"(1, 1)",0,1
32,"(0, 2)",0,1
...,...,...,...
1312,"(10, 12)",0,1
1313,"(10, 12)",1,1
1328,"(11, 12)",0,1
1329,"(11, 12)",1,1


In [3]:
preflop_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rank_set   1352 non-null   object
 1   suited     1352 non-null   int64 
 2   opponents  1352 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 31.8+ KB


In [4]:
from scripts.monte_carlo_methods import compute_win_prob

# using the hands_df, calculate winning odds
hands_df = pd.read_csv('../data/hands.csv')

hands_df['hole1_rank'] = hands_df['hole1'].apply(Card.get_rank_int)
hands_df['hole2_rank'] = hands_df['hole2'].apply(Card.get_rank_int)
hands_df['hole1_suit'] = hands_df['hole1'].apply(Card.get_suit_int)
hands_df['hole2_suit'] = hands_df['hole2'].apply(Card.get_suit_int)
hands_df['suited'] = hands_df['hole1_suit'] == hands_df['hole2_suit']
hands_df['rank_set'] = hands_df.apply(lambda x: frozenset([x['hole1_rank'], x['hole2_rank']]), axis=1)
preflop_df['rank_set'] = preflop_df['rank_set'].apply(lambda x: frozenset(x))


evaluator = Evaluator()
deck = Deck()

# Merge hands_df with preflop_df on rank_set
merged = preflop_df.merge(hands_df, on=['rank_set', 'suited'])
# 
grouped = merged.groupby(['rank_set', 'suited', 'opponents']).apply(lambda row: compute_win_prob(row, deck, evaluator))
grouped

rank_set  suited  opponents
(0)       0       1            0.523490
                  2            0.337808
                  3            0.248322
                  4            0.201342
                  5            0.178971
                                 ...   
(12)      0       4            0.574949
                  5            0.527721
                  6            0.459959
                  7            0.400411
                  8            0.361396
Length: 1352, dtype: float64

In [5]:
# Turn Series into DataFrame
grouped = grouped.reset_index(name='win_prob')

# Rename columns to match preflop_df
grouped = grouped.rename(columns={'suited_pre': 'suited'})

In [6]:
grouped[grouped['opponents'] == 1]

,rank_set,suited,opponents,win_prob
0,(0),0,1,0.523490
8,"(9, 10)",0,1,0.565775
16,"(9, 10)",1,1,0.597270
24,"(8, 10)",0,1,0.602799
32,"(8, 10)",1,1,0.647651
...,...,...,...,...
1312,"(3, 6)",1,1,0.454545
1320,"(2, 6)",0,1,0.417722
1328,"(2, 6)",1,1,0.406143
1336,(8),0,1,0.751073


In [9]:
def preflop_win_prob(r1, r2, suited, opponents):
    rank_set = frozenset([r1, r2])
    return grouped[(grouped['rank_set'] == rank_set) & (grouped['suited'] == suited) & (grouped['opponents'] == opponents)]['win_prob']

In [24]:
preflop_win_prob(12,12 , 0, 8)

1351    0.361396
Name: win_prob, dtype: float64